In [0]:
from pyspark.sql.functions import col,when
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
import requests
from pyspark.sql.functions import when, col, array, concat_ws
import pandas as pd


In [0]:
api_key = "408a589f7f2d505ff09be001d178006b" 
base_url = "http://api.openweathermap.org/data/2.5/weather"
city = "Seoul"
url = f"{base_url}?q={city}&appid={api_key}"
response = requests.get(url)
print(response.json())

{'coord': {'lon': 126.9778, 'lat': 37.5683}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 272.81, 'feels_like': 272.81, 'temp_min': 272.81, 'temp_max': 272.81, 'pressure': 1023, 'humidity': 39, 'sea_level': 1023, 'grnd_level': 1013}, 'visibility': 10000, 'wind': {'speed': 1.04, 'deg': 171, 'gust': 0.98}, 'clouds': {'all': 0}, 'dt': 1735661644, 'sys': {'country': 'KR', 'sunrise': 1735685212, 'sunset': 1735719833}, 'timezone': 32400, 'id': 1835848, 'name': 'Seoul', 'cod': 200}


In [0]:

Weather_list1 = []
# Your OpenWeather API key
api_key = "408a589f7f2d505ff09be001d178006b"  # Replace with your actual API key

# List of cities to get weather for
cities = ["London", "New York", "Tokyo", "Mumbai", "Sydney", "Singapore", "Seoul", "Beijing", "Cape Town","Moscow", "Paris", "Venice", "Berlin", "Ottawa", "Canberra", "Brasília", "Madrid", "Rome", "Bangkok","Nairobi", "Buenos Aires", "Wellington", "Oslo", "Cairo", "Helsinki","Abu Dhabi"]


# Base URL for the OpenWeather API
base_url = "http://api.openweathermap.org/data/2.5/weather"

# Function to fetch weather for a single city
def get_weather(city, api_key):
    try:
        # Construct the request URL
        url = f"{base_url}?q={city}&appid={api_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            # Extract useful information
            weather_info = [
                data["name"],
                round(data["main"]["temp"] - 273.15, 2),  # Convert Kelvin to Celsius
                data["weather"][0]["description"],
                data["main"]["humidity"],
                data["wind"]["speed"],
                round(data["main"]["temp_min"] - 273.15, 2),
                round(data["main"]["temp_max"] - 273.15, 2),
                data["main"]["sea_level"]
                
                
            ]
            return weather_info
        else:
            return {"city": city, "error": f"Error {response.status_code}: {response.text}"}
    except Exception as e:
        return {"city": city, "error": str(e)}

# Fetch weather for all cities
for city in cities:
    weather = get_weather(city, api_key)
    Weather_list1.append(weather)


print(Weather_list1)

df_new = pd.DataFrame(Weather_list1, columns=['City', 'temperature', 'weather', 'humidity', 'wind speed', 'Min_temperature', 'Max_temperature', 'sea_level'])

df_2 = df_new.copy()
   
# Normalize columns directly using a loop
columns_to_normalize = ['temperature', 'humidity', 'wind speed']
normalized_columns = []

# Loop through each column to normalize and store normalized column names
for column in columns_to_normalize:
    min_val = df_new[column].min()
    max_val = df_new[column].max()
    normalized_col_name = f"{column}_normalized"
    df_new[normalized_col_name] = (df_new[column] - min_val) / (max_val - min_val)
    normalized_columns.append(normalized_col_name)

# Calculate Comfort Index using a loop
comfort_index = []
for _, row in df_new.iterrows():
    temperature_score = 0.4 * (1 - abs(row['temperature_normalized'] - 0.5))
    humidity_score = 0.3 * (1 - abs(row['humidity_normalized'] - 0.5))
    wind_speed_score = 0.2 * (1 - row['wind speed_normalized'])
    weather_condition_score = 0.1 if "clear" in row['weather'].lower() else 0.05
    comfort_index.append(
        temperature_score + humidity_score + wind_speed_score + weather_condition_score
    )

# Assign the Comfort Index to the DataFrame
df_new['Comfort_Index'] = comfort_index

df_new_final = df_new[['City','Comfort_Index']].sort_values(by='Comfort_Index',ascending=False)




[['London', 6.8, 'heavy intensity rain', 90, 7.2, 5.9, 7.95, 1006], ['New York', 8.24, 'mist', 88, 5.66, 6.88, 8.98, 1000], ['Tokyo', 6.49, 'few clouds', 43, 3.09, 5.88, 8.08, 1017], ['Mumbai', 27.99, 'haze', 51, 2.57, 27.99, 27.99, 1013], ['Sydney', 23.43, 'overcast clouds', 82, 8.23, 21.79, 24.28, 1014], ['Singapore', 26.75, 'broken clouds', 83, 4.63, 26.61, 26.97, 1011], ['Seoul', -0.24, 'clear sky', 80, 2.57, -1.22, -0.24, 1024], ['Beijing', -3.06, 'broken clouds', 32, 2.02, -3.06, -3.06, 1029], ['Cape Town', 24.7, 'clear sky', 47, 7.72, 23.84, 26.72, 1013], ['Moscow', -0.42, 'light snow', 98, 6.31, -1.76, -0.35, 1000], ['Paris', 7.4, 'overcast clouds', 88, 8.23, 6.32, 8.02, 1018], ['Venice', 19.71, 'clear sky', 94, 0.45, 18.35, 19.99, 1020], ['Berlin', 7.0, 'scattered clouds', 77, 11.62, 6.66, 7.77, 1008], ['Ottawa', 2.01, 'light rain', 89, 5.66, 0.91, 2.77, 1001], ['Canberra', 17.09, 'broken clouds', 82, 3.6, 16.38, 18.07, 1016], ['Brasília', 22.51, 'thunderstorm', 83, 2.57, 22.0

In [0]:
df_new_final

,City,Comfort_Index
16,Madrid,0.866170
17,Rome,0.816110
11,Venice,0.775101
18,Bangkok,0.760388
14,Canberra,0.756986
6,Seoul,0.732631
21,Wellington,0.724986
2,Tokyo,0.722922
23,Cairo,0.715286
15,Brasília,0.704887
